# Modeling

## Data Preprocesses
 1.Binary encoding to "1" helpful, "0" helpfulness
  <br>2.Split files into training and testing sets 70% , 30%
  <br>3.Creat Sample pipeline for text feature extraction and evaluation
  <br>4.Investigate different algorithms: Naive Bayes, Logistic Regression, Random Forest and Ada-boost classifier.  Use a grid search with cross validation to optimize on the hyperparameters of each algorithm.
  <br>5.k-fold cross-validation strategy all classficaiton algorthms to fully leaverge the entire training set.
No validation data require 

In [32]:
%matplotlib inline
import numpy as np 
import pandas as pd 
from pandas import Series, DataFrame, datetime
from datetime import datetime, timedelta, time
from sklearn.model_selection import train_test_split
import nltk
import string
import matplotlib.pyplot as plt
RAN_STATE = 42

In [33]:
df = pd.read_csv('amazon_clean.csv')
df.shape

(568412, 21)

In [133]:
##include reviews that have more than 5 helpfulness data point only
final_df= df[(df.HelpfulnessDenominator >= 5) & (df.year>=2008)]
final_df.shape

(56059, 21)

In [134]:
# for model# 
df_model=final_df[['Helpful','text_Clean2']]
df_model.shape

(56059, 2)

In [135]:
df_model['Helpful'].value_counts()/len(df_model)

1    0.675467
0    0.324533
Name: Helpful, dtype: float64

In [136]:
# create X, y for upsampled training and testing
X_train, X_test, y_train, y_test= train_test_split(df_model['text_Clean2'],
                                                   df_model['Helpful'], 
                                                   test_size=0.3, 
                                                   random_state=RAN_STATE)
print("X_train shape: {}".format(X_train.shape))
print("y_train shape: {}".format(y_train.shape))
print("X_test shape: {}".format(X_test.shape))
print("y_test shape: {}".format(y_test.shape))

X_train shape: (39241,)
y_train shape: (39241,)
X_test shape: (16818,)
y_test shape: (16818,)


#### Total 67,422  data point for modeling 

### Train model

In [137]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.pipeline import Pipeline
from sklearn.utils import shuffle
from sklearn.svm import LinearSVC
from sklearn import metrics
from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split
from sklearn.metrics import make_scorer
from sklearn.metrics import accuracy_score, precision_score, recall_score
from sklearn.metrics import roc_auc_score
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.externals import joblib

### Baseline model :Naive Bayes 

In [138]:
training_b, testing_b = train_test_split(df_model, test_size=0.3, random_state=RAN_STATE)
print("X_train shape: {}".format(training_b.shape))
print("y_train shape: {}".format(testing_b.shape))

X_train shape: (39241, 2)
y_train shape: (16818, 2)


In [139]:
count_vect = CountVectorizer(min_df = 0.01, ngram_range = (1, 1))
X_train_counts = count_vect.fit_transform(training_b['text_Clean2'])

tfidf_transformer = TfidfTransformer()
X_train_tfidf = tfidf_transformer.fit_transform(X_train_counts)

X_new_counts = count_vect.transform(testing_b['text_Clean2'])
X_test_tfidf = tfidf_transformer.transform(X_new_counts)

y_train_b = training_b["Helpful"]
y_test_b = testing_b["Helpful"]

prediction = dict()

x_tr = X_train_tfidf.toarray()
x_te = X_test_tfidf.toarray()

In [140]:
model_GNB = GaussianNB()
model_GNB.fit(x_tr,y_train_b)
train_accuracy = model_GNB.score(x_tr, y_train_b)
test_accuracy = model_GNB.score(x_te, y_test_b)
print('Gaussian Naive Bayes')
print('Accuracy on the training set: {}'.format(train_accuracy))
print('Accuracy on the test set: {}'.format(test_accuracy))

Gaussian Naive Bayes
Accuracy on the training set: 0.699854743762901
Accuracy on the test set: 0.6900939469615888


In [141]:
joblib.dump(model_GNB,'GNB_clf_est.pkl')

['GNB_clf_est.pkl']

 + max_features = n : only considers the top n words orderd by term frequency
 + min_df = n : ignores words with a document frequency below n
 + max_df = n : ignores words with a document frequency above n
 + TF*IDF = Term Frequency * Inverse Document Frequency<Br>
 + stop_words =  ignores common words like  'the' ,  'that' ,  'which' , etc. You’ll need to define in a list what words you want to include. There are lists of common stop words available online, the NLTK library also has a list of stop words built into it.


## Sample pipeline for text feature extraction 

In [143]:
from sklearn.model_selection import StratifiedKFold
n_jobs=-1
n_folds=5
skfold = StratifiedKFold(n_splits=n_folds,random_state=RAN_STATE, shuffle=False)

In [144]:
#construct pipeline
pipeline1 = Pipeline([
    ('vect', CountVectorizer(stop_words = 'english')),
    ('tfidf', TfidfTransformer()),   
    ('clf', LogisticRegression(random_state=RAN_STATE))
                         ])
pipeline2 = Pipeline([
    ('vect', CountVectorizer(stop_words = 'english')),
    ('tfidf', TfidfTransformer()),   
    ('clf', LinearSVC(random_state=RAN_STATE))
                        ])
pipeline3 = Pipeline([
    ('vect', CountVectorizer(stop_words = 'english')),
    ('tfidf', TfidfTransformer()),   
    ('clf', DecisionTreeClassifier(random_state=42))
                        ])
pipeline4 = Pipeline([
    ('vect', CountVectorizer(stop_words = 'english')),
    ('tfidf', TfidfTransformer()),   
    ('clf', RandomForestClassifier(random_state=RAN_STATE))
                        ])
pipeline5= Pipeline([
    ('vect', CountVectorizer(stop_words = 'english')),
    ('tfidf', TfidfTransformer()),   
    ('clf',  AdaBoostClassifier(random_state=RAN_STATE))
                        ])

# tune parameter 
parameters1 = {'vect__ngram_range': [(1, 1),(1, 2),(1, 3)], #unigram, bigram, trigram
                   'vect__max_df': (0.25, 0.5, 0.75),
                   'clf__C': (np.logspace(-5, 1, 5))}

parameters2 = {'vect__ngram_range': [(1, 1), (1, 2),(1, 3)],
                   'vect__max_df': (0.25, 0.5, 0.75),  
                   'clf__C': (np.logspace(-5, 1, 5))}

parameters3 = {'vect__ngram_range': [(1, 1), (1, 2),(1, 3)],
                   'vect__max_df': (0.25, 0.5, 0.75),
                   'clf__max_depth': (3, 4, 5)}
    
parameters4 = {'vect__ngram_range': [(1, 1), (1, 2),(1, 3)],
                   'vect__max_df': (0.25, 0.5, 0.75),
                   'clf__n_estimators': [300, 500],
                   'clf__max_depth': (3, 4, 5)}
    
parameters5 = {'vect__ngram_range': [(1, 1), (1, 2),(1, 3)],
                   'vect__max_df': (0.25, 0.5, 0.75),
                   'clf__n_estimators': [300, 500]}    
    
#  gridsearch for each classfier 
gs_lr = GridSearchCV(estimator=pipeline1,
                     param_grid=parameters1,
                     n_jobs=-1,
                     scoring = 'accuracy', 
                     cv = skfold)

gs_SVC = GridSearchCV(estimator=pipeline2,
                     param_grid=parameters2,
                     n_jobs=-1,
                     scoring = 'accuracy', 
                     cv = skfold)

gs_DS = GridSearchCV(estimator=pipeline3,
                     param_grid=parameters3,
                     n_jobs=-1,
                     scoring = 'accuracy', 
                     cv = skfold)

gs_RF = GridSearchCV(estimator=pipeline4,
                     param_grid=parameters4,
                     n_jobs=-1,
                     scoring = 'accuracy', 
                     cv = skfold)

gs_Adb = GridSearchCV(estimator=pipeline5,
                     param_grid=parameters5,
                     n_jobs=-1,
                     scoring = 'accuracy', 
                     cv = skfold)

# List of pipelines for ease of iteration
grids = [gs_lr, gs_SVC, gs_DS, gs_RF, gs_Adb]

# Dictionary of pipelines and classifier types for ease of reference
grid_dict = {0: 'Logistic Regression', 
             1: 'Linear Support Vector Machine ', 
             2: 'Decision Tree', 
             3: 'Random Forest', 
             4: 'AdaBoostClassifier'}

# Fit the grid search objects
print('Performing model optimizations...')
for idx, gs in enumerate(grids):
    print('\nEstimator: %s' % grid_dict[idx])
    gs.fit(X_train, y_train)

# Best params
    print('Best params: %s' % gs.best_params_)
# Best training data accuracy
    print('Best training accuracy: %.2f' % gs.best_score_)
# Predict on test data with best params
    y_pred = gs.predict(X_test)

# Test data accuracy of model with best params
    print('Test set accuracy score for best params: %.2f ' % accuracy_score(y_test, y_pred))

Performing model optimizations...

Estimator: Logistic Regression
Best params: {'clf__C': 10.0, 'vect__max_df': 0.5, 'vect__ngram_range': (1, 2)}
Best training accuracy: 0.83
Test set accuracy score for best params: 0.84 

Estimator: Linear Support Vector Machine 
Best params: {'clf__C': 10.0, 'vect__max_df': 0.5, 'vect__ngram_range': (1, 3)}
Best training accuracy: 0.83
Test set accuracy score for best params: 0.84 

Estimator: Decision Tree
Best params: {'clf__max_depth': 5, 'vect__max_df': 0.5, 'vect__ngram_range': (1, 2)}
Best training accuracy: 0.68
Test set accuracy score for best params: 0.68 

Estimator: Random Forest
Best params: {'clf__max_depth': 5, 'clf__n_estimators': 500, 'vect__max_df': 0.5, 'vect__ngram_range': (1, 3)}
Best training accuracy: 0.68
Test set accuracy score for best params: 0.67 

Estimator: AdaBoostClassifier
Best params: {'clf__n_estimators': 500, 'vect__max_df': 0.5, 'vect__ngram_range': (1, 2)}
Best training accuracy: 0.76
Test set accuracy score for b

In [145]:
gs_lr.fit(X_train,y_train)         
joblib.dump(gs_lr, 'logsitc_clf.pkl') 

['logsitc_clf.pkl']

In [146]:
gs_SVC.fit(X_train,y_train)
joblib.dump(gs_SVC, 'SVM_clf.pkl')

['SVM_clf.pkl']

In [147]:
gs_RF.fit(X_train,y_train)
joblib.dump(gs_RF, 'rf_clf.pkl')

['rf_clf.pkl']

In [ ]:
gs_DS.fit(X_train, y_train)
joblib.dump(gs_DS, 'ds_clf.pkl')

['ds_clf.pkl']

In [ ]:
gs_Adb.fit(X_train,y_train)
joblib.dump(gs_Adb, 'adb_clf.pkl')

In [ ]:
gnb_clf_est1=joblib.load('GNB_clf_est.pkl') 
log_clf_est1=joblib.load('logsitc_clf.pkl')
SVM_clf_est1=joblib.load('SVM_clf.pkl')
rf_clf_est1=joblib.load('rf_clf.pkl') 
DS_clf_est1=joblib.load('ds_clf.pkl')
adb_clf_est1=joblib.load('adb_clf.pkl')

In [ ]:
def roc_curve_plot(model, X, y,label=None):
    # make sure positive class prediction is in the second column of binary prediction
    if label=='Neural Network':
        y_score = model.predict_proba(X)[:,0]
    else:
        y_score = model.predict_proba(X)[:,1]
    
    # generate ROC curve data
    roc = roc_curve(y, y_score)
    
    plt.plot(roc[0], roc[1], label=label)
    plt.plot([0,1],[0,1], 'k--')
    plt.axis([0,1,0,1])
    plt.xlabel('FPR')
    plt.ylabel('TPR')
    
    roc_score = auc(roc[0],roc[1])
    print('AUC score of %s is %.2f.' % (label, roc_score))

In [ ]:
from sklearn.metrics import roc_curve, auc

In [ ]:
# plot ROC curve for each model on training set
roc_curve_plot(gnb_clf_est1, x_tr, y_train_b, label='Gaussian Naive Bayes')
# roc_curve_plot(SVM_clf_est1, X_train, y_train, label='SVM')
roc_curve_plot(log_clf_est1, X_train, y_train, label='Logistic Regression')
roc_curve_plot(DS_clf_est1, X_train, y_train, label='Decision Tree')
roc_curve_plot(rf_clf_est1,X_train, y_train, label='Random Forest')
roc_curve_plot(adb_clf_est1, X_train, y_train, label='AdaBoost')
plt.title('ROC Curves on Training Set')
plt.legend(loc='lower right')        

plt.show()

In [ ]:
# plot ROC curve for each model on testing set
roc_curve_plot(nb_clf_est1, x_te, y_test_b, label='Gaussian Naive Bayes')
# roc_curve_plot(SVM_clf_est1, X_test, y_test, label='SVM')
roc_curve_plot(log_clf_est1, X_test, y_test, label='Logistic Regression')
roc_curve_plot(DS_clf_est1, X_test, y_test, label='Decision Tree')
roc_curve_plot(rf_clf_est1,X_test, y_test, label='Random Forest')
roc_curve_plot(adb_clf_est1, X_test, y_test, label='AdaBoost')
plt.title('ROC Curves on Testing Set')
plt.legend(loc='lower right')

## Model Performnace Evaluation